In [ ]:
# Cell 1: Environment & Path Setup
import os
import sys
from pathlib import Path

# --- PSC SPECIFIC CONFIG ---
# Force KaggleHub to download to current directory (likely Scratch)
# instead of ~/.cache (Home) which has a small quota.
os.environ['KAGGLEHUB_CACHE_DIR'] = os.getcwd()

# Clone Repo if needed
if not os.path.exists("lip-sync-dl-f25"):
    !git clone https://github.com/MUKAMAFrancois/lip-sync-dl-f25.git
    %cd lip-sync-dl-f25
else:
    %cd lip-sync-dl-f25
    !git pull

# Install requirements
print("📦 Installing Dependencies...")
!pip install -r requirements.txt
!pip install kagglehub face-alignment pytorch-fid

# Setup Wav2Lip (Models)
!python setup_wav2lip.py

In [ ]:
# Cell 2: Data Download (Safe for HPC)
import kagglehub
import shutil
from pathlib import Path

print("⬇️ Downloading Dataset via KaggleHub...")
# This will now download to ./dataset/... because of the env var we set
path = kagglehub.dataset_download("francoismukama/muavic-german-sample")
print(f"✅ Data downloaded to: {path}")

# Link Data to Expected Path
TARGET_DIR = Path("data/german")
os.makedirs(TARGET_DIR, exist_ok=True)

source_video_dir = Path(path) / "mtedx/video/de"

# Move/Link the data
for split in ["train", "val", "test"]:
    src = source_video_dir / split
    dst = TARGET_DIR / split
    if src.exists():
        if dst.exists(): 
            shutil.rmtree(dst)
        # Use Copy instead of Symlink to avoid permission issues across filesystems
        shutil.copytree(src, dst)
        print(f"📂 Copied {split} -> {dst}")

In [ ]:
# Cell 3: Preprocessing (Affine Alignment)
# We use the updated preprocess_training_data.py provided earlier
print("🎬 Starting Preprocessing (Face Alignment & Cropping)...")
!python preprocess_training_data.py --data_root data/german --no-zip

In [ ]:
# Cell 4: Training
# WARNING: On PSC Login Nodes, do NOT run this cell for long.
# Use the Slurm script below for actual 2-day training.
# This cell is for debugging/verification.
print("🔥 Starting Training...")
!python training/train.py

In [ ]:
# Cell 5: Inference Generation
import torch
import cv2
import numpy as np
import os
from pathlib import Path
from tqdm import tqdm
from models import Wav2Lip
from training.hparams import hparams
import audio

def run_inference(checkpoint_path, data_root, output_root):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # 1. Load Model
    print(f"🔄 Loading model: {checkpoint_path}")
    model = Wav2Lip().to(device)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    s = checkpoint["state_dict"]
    new_s = {}
    for k, v in s.items():
        new_s[k.replace('module.', '')] = v
    model.load_state_dict(new_s)
    model.eval()
    
    # 2. Setup
    data_root = Path(data_root)
    output_root = Path(output_root)
    output_root.mkdir(parents=True, exist_ok=True)
    
    # Get Test Videos (from filelist)
    with open("data/german/filelists/test.txt", 'r') as f:
        # Filelist contains "VideoName", we need to find the folder
        # Our preprocess script saves output folders named by VideoName
        test_videos = list(set([x.strip() for x in f.readlines()]))

    print(f"🎥 Generating videos for {len(test_videos)} test clips...")

    for vid_name in tqdm(test_videos):
        # Path to preprocessed frames (Ground Truth)
        gt_path = data_root / vid_name
        if not gt_path.exists(): continue
        
        frames = sorted(list(gt_path.glob("*.jpg")))
        audio_path = gt_path / "audio.wav"
        
        if len(frames) < 5: continue
        
        # Output folder
        vid_out_dir = output_root / vid_name
        vid_out_dir.mkdir(exist_ok=True)
        
        # Load Audio
        try:
            wav = audio.load_wav(str(audio_path), 16000)
            mel = audio.melspectrogram(wav)
        except: continue

        # Simple Batch Inference Logic
        mel_chunks = []
        mel_step_size = 16
        
        # Prepare batches (Simplified for demo)
        # In production, process in batches of 8 or 16
        with torch.no_grad():
            for i, fname in enumerate(frames):
                # 1. Prepare Input Image (Masked)
                img = cv2.imread(str(fname))
                img = cv2.resize(img, (256, 256))
                
                # Mask bottom half
                masked_img = img.copy()
                masked_img[256//2:] = 0
                
                # Ref Image (Random)
                ref_img = img # Simply using self as ref for basic test
                
                # Concatenate
                img_batch = np.concatenate([masked_img, ref_img], axis=2)
                img_batch = img_batch.transpose(2, 0, 1) / 255.0
                img_batch = torch.FloatTensor(img_batch).unsqueeze(0).to(device)
                
                # Audio Slice
                # (Complex slicing logic omitted for brevity, using dummy/center slice)
                # Ideally import the exact slicing from data_loader.py
                mel_batch = torch.FloatTensor(mel[:, :16]).unsqueeze(0).to(device) 
                
                # Forward
                pred = model(img_batch, mel_batch)
                
                # Save
                pred = pred.cpu().numpy().transpose(0, 2, 3, 1) * 255.
                cv2.imwrite(str(vid_out_dir / fname.name), pred[0].astype(np.uint8))

# Find latest checkpoint
checkpoints = sorted(Path("checkpoints").glob("*.pth"), key=os.path.getmtime)
if checkpoints:
    latest = checkpoints[-1]
    # Generate Frames
    run_inference(latest, "data/german/preprocessed", "results/gen")
else:
    print("❌ No checkpoint found.")

In [ ]:
# Cell 6: Run Full Evaluation
# Compare Ground Truth (data/german/preprocessed) vs Generated (results/gen)
!python evaluate.py --gt_path data/german/preprocessed --gen_path results/gen